In [1]:
# Created by Alexander Bolejack & Austin Roddy
# Illinois Institute of Technology
# Intllgnc Txt Analys Knwldg Mgm (CS-481-01), Spring 2022
# Programming Assignment 2 - Final Project

In [2]:
# Inport the necessary libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as sw
import re
from nltk.probability import FreqDist

In [3]:
# dowload corpori for nltk (used later)
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [4]:
# Import the data, and drop the unnecessary columns, remove non-alphabetical characters(first pass)
full_dataset_df = pd.read_csv ("dataset.csv")
full_dataset_df.drop('app_id', inplace=True, axis=1)
full_dataset_df.drop('app_name', inplace=True, axis=1)
full_dataset_df.drop('review_votes', inplace=True, axis=1)
full_dataset_df.rename(columns={'review_text': 'text', 'review_score': 'label'}, inplace=True)

In [5]:
# Remove duplicate messages in entries in the dataset
duplicates = full_dataset_df.duplicated(subset='text', keep='first')
full_dataset_df = full_dataset_df.drop(full_dataset_df[duplicates].index)


# I wouldnt trust this function HAHAHAHA it deletes 1.6 million rows

In [6]:
# Normalize the labels if -1 switch to 0
full_dataset_df['label'] = full_dataset_df['label'].replace(-1, 0)

In [7]:
# Randomize the dataframe
randomized_full_dataset_df = full_dataset_df.sample(frac=1).reset_index(drop=True)

In [8]:
# Split the data into training and testing sets (90% training, 10% testing) (and randomize... again...? *shrug*)
training_dataset_df = randomized_full_dataset_df.sample(frac=0.9,random_state=200)
testing_dataset_df = randomized_full_dataset_df.drop(training_dataset_df.index)

In [9]:
# Create both pos and neg versions of training and testing sets (4 sets)
training_pos_dataset_df = training_dataset_df[training_dataset_df['label'] == 1]
training_neg_dataset_df = training_dataset_df[training_dataset_df['label'] == 0]
testing_pos_dataset_df = testing_dataset_df[testing_dataset_df['label'] == 1]
testing_neg_dataset_df = testing_dataset_df[testing_dataset_df['label'] == 0]

In [10]:
# Convert ALL text rows to string, if not already string (this was causing issues which is why im now converting to string)
# Ignore the output warnings, or better yet, disable them. >:) (this is a bit of a hack, but it works)
training_pos_dataset_df['text'] = training_pos_dataset_df['text'].astype(str)
training_neg_dataset_df['text'] = training_neg_dataset_df['text'].astype(str)
testing_pos_dataset_df['text'] = testing_pos_dataset_df['text'].astype(str)
testing_neg_dataset_df['text'] = testing_neg_dataset_df['text'].astype(str)

C:\Users\Alex\AppData\Local\Temp/ipykernel_27280/4058182508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_pos_dataset_df['text'] = training_pos_dataset_df['text'].astype(str)
C:\Users\Alex\AppData\Local\Temp/ipykernel_27280/4058182508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_neg_dataset_df['text'] = training_neg_dataset_df['text'].astype(str)
C:\Users\Alex\AppData\Local\Temp/ipykernel_27280/4058182508.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [11]:
# Lowercase ALL sets
# Ignore the output warnings, or better yet, disable them. >:) (this is a bit of a hack, but it works)
training_pos_dataset_df.loc[:,'text'] = training_pos_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
training_neg_dataset_df.loc[:,'text'] = training_neg_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
testing_pos_dataset_df.loc[:,'text'] = testing_pos_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
testing_neg_dataset_df.loc[:,'text'] = testing_neg_dataset_df.loc[:,'text'].apply(lambda x: x.lower())

C:\Users\Alex\anaconda3\envs\VSCode\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [12]:
# Dataframe rows to lists
training_pos_dataset_list = training_pos_dataset_df['text'].tolist()
training_neg_dataset_list = training_neg_dataset_df['text'].tolist()
testing_pos_dataset_list = testing_pos_dataset_df['text'].tolist()
testing_neg_dataset_list = testing_neg_dataset_df['text'].tolist()

In [13]:
# Remove all non-alphabetical characters
training_pos_dataset_list_alpha_cleaned = []
training_neg_dataset_list_alpha_cleaned = []
testing_pos_dataset_list_alpha_cleaned = []
testing_neg_dataset_list_alpha_cleaned = []

for i in range(len(training_pos_dataset_list)):
    training_pos_dataset_list_alpha_cleaned.append(''.join([ch for ch in training_pos_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(training_neg_dataset_list)):
    training_neg_dataset_list_alpha_cleaned.append(''.join([ch for ch in training_neg_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(testing_pos_dataset_list)):
    testing_pos_dataset_list_alpha_cleaned.append(''.join([ch for ch in testing_pos_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(testing_neg_dataset_list)):
    testing_neg_dataset_list_alpha_cleaned.append(''.join([ch for ch in testing_neg_dataset_list[i] if ch.isalpha() or ch == ' ']))

# wow i cant believe that worked?!

In [14]:
# Tokenize each list
training_pos_dataset_list_alpha_cleaned_tokenized = []
training_neg_dataset_list_alpha_cleaned_tokenized = []
testing_pos_dataset_list_alpha_cleaned_tokenized = []
testing_neg_dataset_list_alpha_cleaned_tokenized = []

for i in range(len(training_pos_dataset_list_alpha_cleaned)):
    training_pos_dataset_list_alpha_cleaned_tokenized += training_pos_dataset_list_alpha_cleaned[i].split()

for i in range(len(training_neg_dataset_list_alpha_cleaned)):
    training_neg_dataset_list_alpha_cleaned_tokenized += training_neg_dataset_list_alpha_cleaned[i].split()

for i in range(len(testing_pos_dataset_list_alpha_cleaned)):
    testing_pos_dataset_list_alpha_cleaned_tokenized += [testing_pos_dataset_list_alpha_cleaned[i].split()]

for i in range(len(testing_neg_dataset_list_alpha_cleaned)):
    testing_neg_dataset_list_alpha_cleaned_tokenized += [testing_neg_dataset_list_alpha_cleaned[i].split()]

In [21]:
# Remove stopwords
training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
stop_words = set(sw.words('english'))


for i in training_pos_dataset_list_alpha_cleaned_tokenized:
    if i not in stop_words:
        training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

for i in training_neg_dataset_list_alpha_cleaned_tokenized:
    if i not in stop_words:
        training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

        

for i in testing_pos_dataset_list_alpha_cleaned_tokenized:
    words = []
    for w in i:
        if w not in stop_words:
            words.append(w)
    testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed += [words]

for i in testing_neg_dataset_list_alpha_cleaned_tokenized:
    words = []
    for w in i:
        if w not in stop_words:
            words.append(w)
    testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed += [words]

In [27]:
# Stemmed each set:
ps = PorterStemmer()

training_pos_dataset_list_alpha_cleaned_stemmed = []
training_neg_dataset_list_alpha_cleaned_stemmed = []


for i in range(len(training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    training_pos_dataset_list_alpha_cleaned_stemmed.append(ps.stem(training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i]))

for i in range(len(training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    training_neg_dataset_list_alpha_cleaned_stemmed.append(ps.stem(training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i]))


In [ ]:
# Stem the test. Seperate from training in case we load the model in but want to run the test data
testing_pos_dataset_list_alpha_cleaned_stemmed = []
testing_neg_dataset_list_alpha_cleaned_stemmed = []
ps = PorterStemmer()

for i in range(len(testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    words = []
    for w in range(len(testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i])):
        words.append(ps.stem(testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i][w]))
    testing_pos_dataset_list_alpha_cleaned_stemmed += [words]


for i in range(len(testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    words = []
    for w in range(len(testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i])):
        words.append(ps.stem(testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i][w]))
    testing_neg_dataset_list_alpha_cleaned_stemmed += [words]


### With clean data now, we can make our Bayes Model
##### For this, we can get work counts over the total number of words for each class
##### Using these values, as well as the probability of the class, we can make our prediction

In [28]:
#Probability of class
probability_pos = len(training_pos_dataset_df)/ len(training_dataset_df)
probability_neg = len(training_neg_dataset_df)/ len(training_dataset_df)

In [29]:
#Get total count of words for each class
pos_denom = len(training_pos_dataset_list_alpha_cleaned_stemmed)
neg_denom = len(training_neg_dataset_list_alpha_cleaned_stemmed)

In [38]:
#Get counts of the words for each class
pos_nom = FreqDist(training_pos_dataset_list_alpha_cleaned_stemmed)
neg_nom = FreqDist(training_neg_dataset_list_alpha_cleaned_stemmed)

In [31]:
#Lets get information about smoothing. Simple laplace +1
pos_vocab_size = len(pos_nom)
neg_vocab_size = len(neg_nom)

In [49]:
def save_model(probability, denom, vocab_size, nom, name):
    with open(name, 'w') as f:
        f.write(str(probability) + '\n')
        f.write(str(denom) + '\n')
        f.write(str(vocab_size) + '\n\n')
        for i in list(nom.items()):
            f.write(str(i[0]) +' ' + str(i[1]) + '\n')

In [50]:
save_model(probability_pos, pos_denom, pos_vocab_size, pos_nom, 'pos_prob.txt')
save_model(probability_neg, neg_denom, neg_vocab_size, neg_nom, 'neg_prob.txt')

In [67]:
def load_model(file):
    model = open(file, 'r')
    Lines = model.readlines()
    count = 0
    wourd_count_dict = {}
    for line in Lines:
        if count == 0:
            class_probability = line
        elif count == 1:
            denominator = line
        elif count == 2:
            vocab_size = line
        else:
            line_split = line.split()
            if len(line_split) == 2:
                wourd_count_dict[line_split[0]] = int(line_split[1])
        count = count + 1

    return float(class_probability), int(denominator), int(vocab_size), wourd_count_dict

### Lets have an inference function

##### Takes in a list of strings, that has aready been tokenized etc,
##### Returns the predicted class, 1 or -1
##### Optional print value, enabled by default

In [72]:
def inference(sentence_precoessed,prob_pos, pos_denom, pos_vocab, pos_nom,prob_neg, neg_denom, neg_vocab, neg_nom, p_print=True):
    pos_value = 1
    neg_value = 1
    for word in sentence_precoessed:
        if word in pos_nom:
            pos_value = pos_value * ( (pos_nom[word]+1) / (pos_denom+pos_vocab) )
        if word in neg_nom:  
            neg_value = neg_value * ( (neg_nom[word]+1) / (neg_denom+neg_vocab) )

    pos_value = pos_value * prob_pos
    neg_value = neg_value * prob_neg

    if p_print:
        print('Positive review prediction: ' + str(pos_value))
        print('Negative review prediction: ' + str(neg_value))

    if pos_value >= neg_value:
        return 1
    else:
        return -1

In [69]:
def normalize_sentence(sentence):
    sentence = sentence.lower()
    new_sentence = []
    new_sentence.append(''.join([ch for ch in sentence if ch.isalpha() or ch == ' ']))
    new_sentence = new_sentence[0].split()

    stop_words = set(sw.words('english'))
    sentence_no_stop_words = []
    for i in new_sentence:
        if i not in stop_words:
            sentence_no_stop_words.append(i)

    sentence_stemmed = []

    for i in range(len(sentence_no_stop_words)):
        sentence_stemmed.append(ps.stem(sentence_no_stop_words[i]))

    return sentence_stemmed

In [78]:
#From a sentence to input
sentence = "How people can fall for this braindead mess of minigames filled with obnoxious cutscenes is beyond me.Do yourself a favor and go watch some yakuza movies instead if you like this setting, I guarantee they're funnier."
sentence_normalized = normalize_sentence(sentence)
print(sentence_normalized)
class_prob_pos, denom_pos, vocab_pos, wourd_dict_pos = load_model('pos_prob.txt')
class_prob_neg, denom_neg, vocab_neg, wourd_dict_neg = load_model('neg_prob.txt')
prediction = inference(sentence_normalized, class_prob_pos, denom_pos, vocab_pos, wourd_dict_pos, class_prob_neg, denom_neg, vocab_neg, wourd_dict_neg)
prediction

['peopl', 'fall', 'braindead', 'mess', 'minigam', 'fill', 'obnoxi', 'cutscen', 'beyond', 'medo', 'favor', 'go', 'watch', 'yakuza', 'movi', 'instead', 'like', 'set', 'guarante', 'theyr', 'funnier']
Positive review prediction: 2.0686892714848466e-82
Negative review prediction: 1.4591353478242677e-74


-1